Артем Жук, 399 группа
### EM for PCA

В данном репозитории реализованы две вариации ЕМ-алогоритма для анализа главных компонент. Статья с описанием выкладок: http://www.machinelearning.ru/wiki/images/7/73/BMMO11_11.pdf

#### EMPCA
Класс EMPCA является реализацией ЕМ алгоритма для нахожения подпространста пространства признаков, порожденного первыми несколькими главными компонентами. Для всей выборки все признаки должны быть известны.  
Эта реализация опирается на достаточно простые выкладки, а кроме того быстрее работает, чем более общая реализация EMPCAM.

In [22]:
from empca import EMPCA
from empca_missing import EMPCAM
import numpy as np
from utils import random_model, gram_schmidt, span_in

Сгенерируем выборку. Метод random_model(N, D, d) генрирует выборку с распределением $W\mathcal{N}(0, I_d) + \mathcal{N}(0, \varepsilon I_D), W\in M(\mathbb{R})^{D\times d}$.  
gram_schmidt(X) применяет алгоритм Грама-Шмидта к строкам Х.

In [3]:
X, W, T = random_model(30, 6, 1)
print("W=\n", W)
print("X..=\n", X[:5])

W=
 [[-1.59626617  9.13118557 -8.23301779 -7.76449735 -3.24425211  6.46349645]]
X..=
 [[ -2.08585544   2.96669156  -3.76675187  -3.27459908  -1.54818168
    2.86508096]
 [ -0.33931591   2.45621328  -3.08450874  -1.56957656  -0.59062461
    1.42178233]
 [ -0.89310368  12.16058825 -10.4973341  -11.59801969  -3.6236066
    9.44584531]
 [ -0.76359673   4.18888379  -3.69179469  -3.6385299   -1.36644908
    3.15850942]
 [ -1.56028055   1.16739967  -0.44367479  -0.49911262  -0.76524781
    0.46165147]]


Создадим экземпляр EMPCA и запустим на нашей выборке. Поле components_ содержит ортнормированный базис из n_components главных векторов.

In [4]:
empca = EMPCA(n_components=1, n_iter=100)
empca.fit(X)
print('basis:\n', empca.components_)
print('original basis:\n', gram_schmidt(W))

basis:
 [[ 0.07617539 -0.56683178  0.50045351  0.46818803  0.19425784 -0.40682819]]
original basis:
 [[-0.09781847  0.55955496 -0.50451564 -0.47580492 -0.19880631  0.39608016]]


Как видим, алгоритм довольно точно определил направляющий вектор прямой, на которой лежат точки. Попробуем для плоскости:

In [14]:
X, W, T = random_model(60, 7, 5)
empca = EMPCA(n_components=1, n_iter=100)
empca.fit(X)
print('basis:\n', empca.components_)
print('original basis:\n', gram_schmidt(W))

basis:
 [[-0.13075886  0.05886138  0.49007968 -0.30300823  0.47582618  0.64283461
   0.08830908]]
original basis:
 [[ 0.72143147 -0.37949328  0.45550436  0.2501199   0.12566343  0.08429369
   0.2063506 ]
 [ 0.30409151 -0.28704947 -0.20609003 -0.76093067 -0.08394842  0.32911956
  -0.29711168]
 [ 0.24934622 -0.32364301 -0.44763801  0.21698083 -0.29957111 -0.63758155
  -0.298945  ]
 [ 0.12632001  0.24533764  0.10892963  0.12740927  0.59925971 -0.04705042
  -0.7310458 ]
 [ 0.51955573  0.75867281 -0.12239662  0.01549499 -0.36579093  0.06965908
   0.02451504]]


На этот раз сложнее проверить совпадение векторов. Однако для проверки корректности нам можно проверять не точное совпадение базисов, а совпадение подпростраств, на них натянутых. Для этого воспользуемся функцией span_in(A, B), проверяющей что строки А лежат в линейной оболочке строк B.

In [15]:
print('span_in: ', span_in(empca.components_, gram_schmidt(W)))

span_in:  True


#### EMPCAM
Этот класс реализует тот же алгоритм, но допускает пропуски в данных. 


Сгененрируем выборку с 20% пропущенных значений:

In [26]:
X, W, T = random_model(100, 10, 1, 0.2)
print("X=\n", X[:5])

X=
 [[             nan  -4.02453916e+00   1.81503374e+00              nan
   -4.96381543e-01   1.16963088e+00   1.60662142e+00   3.21152962e+00
   -8.22836820e-01  -1.71741760e+00]
 [ -6.88219945e-01              nan   2.01782655e+00  -2.49448420e+00
   -3.37794152e-01   3.19029873e+00   9.41216998e-01              nan
   -3.88224414e-01  -4.56005017e+00]
 [ -1.16785928e+00  -6.14639392e+00              nan  -2.51850289e+00
   -2.80494259e-03   2.29252007e+00   8.12264959e-01              nan
    8.36738923e-01  -4.40353040e+00]
 [  2.08146990e+00   7.34885118e+00  -2.90916146e+00   3.28075490e+00
   -3.69614366e-01  -3.78734262e+00              nan  -6.94128832e+00
   -1.57021504e+00   5.17155544e+00]
 [ -3.24066396e+00  -1.08150815e+01   5.19114164e+00  -5.55890118e+00
   -2.38232461e-01              nan              nan              nan
    3.14026420e-01  -6.54140418e+00]]


И применим к ней алгоритм: 

In [29]:
empcam = EMPCAM(n_components=1, n_iter=100)
empcam.fit(X)
A = empcam.components_
B = gram_schmidt(W)
print('found basis: ', A)
print('original basis: ', B)
d = A[0] - B[0]
print('A - B: ', d)
print('diff = ', np.linalg.norm(d))

found basis:  [[-0.15412392 -0.51197516  0.24699145 -0.2499966  -0.01244379  0.25434507
   0.1821042   0.55929992  0.03444697 -0.42262628]]
original basis:  [[-0.15827637 -0.5522364   0.23916262 -0.22324195 -0.02316006  0.23398859
   0.16836937  0.57186045  0.03310966 -0.38883402]]
A - B:  [ 0.00415245  0.04026124  0.00782883 -0.02675465  0.01071628  0.02035648
  0.01373482 -0.01256053  0.00133731 -0.03379226]
diff =  0.0665932052955


Как видим, нам удалось довольно точно восстановить исходную зависимость. 